In [1]:
#Dependencies
import pandas as pd
from pathlib import Path
import json
import geopandas as gpd

In [2]:
#Create and read file path

housing_data_to_load = Path("Resources/CA_housing_prices.csv")
housing_data = pd.read_csv(housing_data_to_load)

gdp_data_to_load = Path("Resources/GDP.csv")
gdp_data = pd.read_csv(gdp_data_to_load)

interest_data_to_load = Path("Resources/FEDFUNDS.csv")
interest_data = pd.read_csv(interest_data_to_load)

unemployment_data_to_load = Path("Resources/UNRATE.csv")
unemployment_data = pd.read_csv(unemployment_data_to_load)

housing_data__fips_to_load = Path("Resources/CA_housing_prices_with_FIPS.csv")
housing_data_fips = pd.read_csv(housing_data__fips_to_load)

In [3]:
housing_merged_df = housing_data.merge(gdp_data, on='Date', how='outer').merge(interest_data, on='Date', how='outer').merge(unemployment_data, on='Date', how='outer')
housing_merged_df = housing_merged_df.dropna() 

In [4]:
# Initialize a dictionary to hold the JSON data
pricing_data = {}

# Iterate over columns (excluding 'Date')
for column in housing_merged_df.columns[1:]:
    date_value_dict = {}
    
    # Iterate over rows to extract date-value pairs
    for index, row in housing_merged_df.iterrows():
        date = row['Date']
        value = row[column]
        date_value_dict[date] = value
    
    # Add the date-value dictionary to the JSON data with the column name as the key
    pricing_data[column] = date_value_dict

# Define the name of the JSON file
pricing_json_file_name = 'Resources/pricing_data.json'

# Open the file in write mode and write the JSON data to it
with open(pricing_json_file_name, 'w') as json_file:
    json.dump(pricing_data, json_file, indent=4)

In [5]:
housing_data_fips['FIPS'] = housing_data_fips['FIPS'].astype(str).str.zfill(5)

In [6]:
# Create an empty dictionary to store the JSON data
date_data = {}

# Iterate over rows
for index, row in housing_merged_df.iterrows():
    date = row["Date"]
    gdp = row["GDP"]
    fedfunds = row["FEDFUNDS"]
    unrate = row["UNRATE"]
    
    # Create a dictionary for each county and its corresponding data
    county_data = {}
    for county in housing_merged_df.columns[1:-3]:  # Adjust the column index range accordingly
        county_name = county
        price = row[county]
        
        # Find the FIPS data for the county using the transposed DataFrame
        fips_data = housing_data_fips[housing_data_fips["County"] == county_name]["FIPS"].values[0]
        
        county_data[county_name] = {
            "Price": price,
            "FIPS": fips_data
        }
    
    # Create the JSON object for the current row
    json_object = {
        "Counties": county_data,
        "GDP": gdp,
        "Interest": fedfunds,
        "Unemployment": unrate
    }
    
    # Add the JSON object to the dictionary with the date as the key
    date_data[date] = json_object

# Specify the file path where you want to save the JSON data
file_path = "Resources/date_data.json"

# Write the JSON data to the file
with open(file_path, 'w') as json_file:
    json.dump(date_data, json_file, indent=4)


In [2]:
# Load your dataset with housing price data
housing_data = pd.read_csv('Resources/county_housingprice.csv')

# Load GeoJSON data for California counties
counties_geojson = gpd.read_file('Resources/california-counties.geojson')

# Merge the datasets 
merged_data = counties_geojson.merge(housing_data, how='left', left_on='name', right_on='county_name')

# Save the merged GeoJSON in a new geojson file.
merged_data.to_file('Resources/new_counties.geojson', driver='GeoJSON')

/Users/andreaaguilar/anaconda3/envs/dev/lib/python3.10/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
